#  CDMS Python Application Programming Interface

## A First Example


The following Python script reads January and July monthly temperature
data from an input dataset, averages over time, and writes the results
to an output file. The input temperature data is ordered (time,
latitude, longitude).

In [ ]:
# Download files needed for examples below
import os.path
if(not os.path.exists("tas_mo.nc")):
    !wget  http://cdat.llnl.gov/cdat/sample_data/tas_mo.nc
    !wget  http://cdat.llnl.gov/cdat/sample_data/sample.nc
    !wget  http://cdat.llnl.gov/cdat/sample_data/ta_ncep_87-6-88-4.nc


### Makes the CDM2S and MV2 modules available.

* MV2 defines arithmetic functions.

In [ ]:
import cdms2, cdat_info
from cdms2 import MV2

### Opens a netCDF file read-only. 
* The result jones is a dataset object.

In [ ]:
jones = cdms2.open('tas_mo.nc')

### Gets the surface air temperature variable. ‘tas’ is the name of the variable in the input dataset. 
* This does not actually read the data."

In [ ]:
tasvar = jones['tas']

### Read all January monthly mean data into a variable jans.
* Variables can be sliced like arrays.
* The slice operator [0::12] means take every 12th slice from dimension 0, starting at index 0 and ending at the last index. 
* If the stride 12 were omitted, it would default to 1. 

**Note:** that the variable is actually 3-dimensional. Since no slice is specified for the second or third dimensions, all values of those 2,3 dimensions are retrieved. The slice operation could also have been written [0::12, : , :]. 


**Also note:** that the same script works for multi-file datasets. CDMS opens the needed data files, extracts the appropriate slices, and concatenates them into the result array."
       

In [ ]:
jans = tasvar[0::12]

### Reads all July data into a masked array julys.

In [ ]:
julys = tasvar[6::12]

### Calculate the average January value for each grid zone.
* Any missing data is handled automatically.

In [ ]:
janavg = MV2.average(jans)

### Set the variable id and long\_name attributes. 
* The id is used as the name of the variable when plotted or written to a file."

In [ ]:
janavg.id = "tas_jan"
janavg.long_name = "mean January surface temperature"


### Calculate the average July value for each grid zone.

In [ ]:
julyavg = MV2.average(julys)
julyavg.id = "tas_jul"
julyavg.long_name = "mean July surface temperature"

### Create a new netCDF output file named "***janjuly.nc***" to hold the results.

Write the January average values to the output file. 
* The variable will have id “tas\_jan” in the file.
* "**write**" is a utility function which creates the variable in the file, then writes data to the variable.
* A more general method of data output is first to create a variable, then set a slice of the variable.

**Note:** that janavg and julavg have the same latitude and longitude information as tasvar. It is carried along with the computations."

In [ ]:
out = cdms2.open('janjuly.nc','w')
out.write(janavg)
out.write(julyavg)


### Set the global attribute "**comment**".
* Close the output file.

In [ ]:
out.comment = "Average January/July from Jones dataset"


In [ ]:
jones.close()
out.close()

Look at the resulting file using ncdump.  The written file follows the CF-1 connvention.

In [ ]:
!ncdump -h janjuly.nc

---

## Cdms2 Module


The cdms2 module is the Python interface to CDMS. The objects and methods in this chapter are made accessible with the command:

In [ ]:
import cdms2

The functions described in this section are not associated with a class.
Rather, they are called as module functions, e.g.,

In [ ]:
file = cdms2.open('clt.nc')

**See Also**: [Cdms Module Functions](https://cdms.readthedocs.io/en/latest/manual/cdms_2.html#cdms-module-functions)

### CdmsObj

Get a list of all external attributes of obj.

In [ ]:
extatts = file.attributes.keys()
print(extatts)

### CoordinateAxis

A CoordinateAxis is a variable that represents coordinate information.
It may be contained in a file or dataset, or may be transient
(memoryresident). Setting a slice of a file CoordinateAxis writes to the file, and referencing a file CoordinateAxis slice reads data from the file. Axis objects are also used to define the domain of a Variable.

CDMS defines several different types of CoordinateAxis objects. See [MV module](https://cdms.readthedocs.io/en/latest/manual/cdms_2.html#id4) documents methods that are common to all CoordinateAxis types. See [HorizontalGrid]( https://cdms.readthedocs.io/en/latest/manual/cdms_2.html#id6) specifies methods that are unique to 1D Axis objects.

In [ ]:
clt=file['clt']
axis=clt.getAxis(2)
print(axis)

### isCircular()

Returns True if the axis has circular topology.

An axis is defined as circular if:
* axis.topology == 'circular', or
* axis.topology is undefined, and the axis is a longitude.

The default cycle for circular axes is 360.0

In [ ]:
print(axis.isCircular())

### mapIntervalExt(interval)

Map a coordinate interval to an index
interval. interval is a tuple having one of the forms:
* (x,y)
* (x,y,indicator)
* (x,y,indicator,cycle)

None or ':'

* where x and y are coordinates indicating the interval [x,y], and:
  * indicator is a two or three-character string, where the first character is 'c' if the interval is closed on the left, 'o' if open, and the second character has the same meaning for the right-hand point. If present, the third character specifies how the interval should be intersected with the axis
  * 'n' - select node values which are contained in the interval
  * 'b' -select axis elements for which the corresponding cell boundary intersects the interval
  * 'e' - same as n, but include an extra node on either side
  * 's' - select axis elements for which the cell boundary is a subset of the interval
  
The default indicator is ‘ccn’, that is, the interval is closed, and nodes in the interval are selected.

If cycle is specified, the axis is treated as circular with the given cycle value.

By default, if axis.isCircular() is true, the axis is treated as circular with a default modulus of 360.0.

An interval of None or ':' returns the full index interval of the axis.

The method returns the corresponding index interval as a 3tuple (i,j,k), where k is the integer stride, and  (i.j) is the half-open index interval (i <= k < j),  (i >= k > j if k < 0), or none if the intersection is empty.

For an axis which is circular (axis.topology == 'circular'), (i,j) is interpreted as follows, where n = len(axis)

if **0 <= i < n and 0 <= j <= n**, the interval does not wrap around the axis endpoint.

otherwise the interval wraps around the axis endpoint.

**see also**: mapinterval, variable.subregion()

In [ ]:
print(axis.mapIntervalExt((-5.0,5.0,'co')))

### CdmsFile

A ``CdmsFile`` is a physical file, accessible via the ``cdunif``
interface. netCDF files are accessible in read-write mode. All other
formats (DRS, HDF, GrADS/GRIB, POP, QL) are accessible read-only.

As of CDMS V3, the legacy cuDataset interface is also supported by
Cdms-Files. See “cu Module”.

### The following reads data for variable ‘clt’, year 1980.

In [ ]:
f = cdms2.open('clt.nc')
x = f('clt', time=('1980-1','1981-1'))

### The following gets the axis named time

In [ ]:
t = f.axes['time']
print("axis name:", t.id)
t = f['time']
print("axis name:", t.id)

## 2.9. MV2 Module

The fundamental CDMS data object is the variable. A variable is comprised of:

    a masked data array, as defined in the NumPy "ma" module.
    a domain is an ordered list of axes and/or grids.
    an attribute dictionary.

MV2 can be imported with the command:

In [ ]:
import MV2

For completeness MV2 provides access to all the "numpy.ma" functions. The functions not listed in the following tables are identical to the corresponding numpy.ma function: **allclose, allequal, common_fill_value, compress, create_mask, dot, e, fill_value, filled, get_print_limit, getmask, getmaskarray, identity, indices, innerproduct, isMV2, isMaskedArray, is_mask, isarray, make_mask, make_mask_none, mask_or, masked, pi, put, putmask, rank, ravel, set_fill_value, set_print_limit, shape, size**. See the documentation at https://github.com/numpy/numpy for a description of these functions.

In [ ]:
import cdms2
fh = cdms2.open("ta_ncep_87-6-88-4.nc")
ta=fh['ta']
print(ta.shape)
data = ta.subRegion(':', (-45.0,45.0,'co'), (0.0, 180.0))
print(data.shape)

or equivalently:

In [ ]:
data = ta.subRegion(latitude=(-45.0,45.0,'co'), longitude=(0.0,180.0))
print(data.shape)

Read all data for ``March, 1980``:

In [ ]:
data = ta.subRegion(time=('1988-3','1988-4','co'))
print(data.shape)

### Selectors

A selector is a specification of a region of data to be selected from a variable. For example, the statement:

In [ ]:
x = ta(time='1988-1-1', level=(1000.0,100.0))
print(x.shape)

means `select the values of variable v for time ‘1979-1-1’ and levels 1000.0 to 100.0 inclusive, setting x to the result.` Selectors are generally used to represent regions of space and time.

The form for using a selector is:

In [ ]:
from cdms2.selectors import Selector
s =  Selector(time='1988-1-1', level=(1000.0,100.0))
result = ta(s)
print(result.shape)

where v is a variable and s is the selector. An equivalent form is:

In [ ]:
result = fh('ta', s)
print(result.shape)

where f is a file or dataset, and ‘varid’ is the string ID of a variable.

A selector consists of a list of selector components. 

For example, the selector:

* **time='1979-1-1', level=(1000.0,100.0)**
 
has two components: time=’1979-1-1’, and level=(1000.0,100.0). This illustrates that selector components can be defined with keywords, using the form:

<div class=note>
    <b>NOTE</b>
    
For the keywords time, level, latitude, and longitude, the selector can be used with any variable. If the corresponding axis is not found, the selector component is ignored. This is very useful for writing general purpose scripts. The required keyword overrides this behavior. These keywords take values that are coordinate ranges or index ranges as defined in See Index and Coordinate Intervals.</div>

Another form of selector components is the positional form, where the component order corresponds to the axis order of a variable. For example:


In [ ]:
x9 = ta(('1988-1-1','1988-2-1'),1000.0)

reads data for the range (‘1988-1-1’,’1988-2-1’) of the first axis, and coordinate value ``1000.0`` of the second axis. Non-keyword arguments of the form(s) listed in Index and Coordinate Intervals are treated as positional. Such selectors are more concise, but not as general or flexible as the other types described in this section.

Selectors are objects in their own right. This means that a selector can be defined and reused, independent of a particular variable. Selectors are constructed using the cdms.selectors.Selector class. The constructor takes an argument list of selector components. For example:

In [ ]:
from cdms2.selectors import Selector
sel = Selector(time=('1988-1-1','1988-2-1'), level=1000.)
x1 = ta(sel)
x2 = ta(sel)


For convenience CDMS provides several predefined selectors, which can be used directly or can be combined into more complex selectors. 

The selectors **time**, **level**, **latitude**, **longitude**, and **required** are equivalent to their keyword counterparts. For example:

In [ ]:
from cdms2 import time, level
x = ta(time('1988-1-1','1988-2-1'), level(1000.))
print(x.shape)

and

are equivalent. Additionally, the predefined selectors **latitudeslice**, **longitudeslice**, **levelslice**, and **timeslice** take arguments ``(startindex, stopindex[, stride])``:

In [ ]:
from cdms2 import timeslice, levelslice
x = ta(timeslice(0,2), levelslice(16,17))
print(x.shape)

Finally, a collection of selectors is defined in module cdutil.region:

In [ ]:
from cdutil.region import *

NH=NorthernHemisphere=domain(latitude=(0., 90.))
SH=SouthernHemisphere=domain(latitude=(-90., 0.))
Tropics=domain(latitude=(-23.4,23.4))
SPZ=AAZ=AntarcticZone=domain(latitude=(-90., -66.6))
                             

Selectors can be combined using the `&` operator, 

or by refining them in the call:

In [ ]:
from cdms2.selectors import Selector
from cdms2 import level
sel2 = Selector(time=('1988-1-1','1988-2-1'))
sel3 = sel2 & level(1000.0)
x1 = ta(sel3)
print(x1.shape)
x2 = ta(sel2, level=1000.0)
print(x2.shape)

### Selector Examples

CDMS provides a variety of ways to select or slice data. In the following examples, variable hus is contained in file ``sample.nc``, and is a function of **(time, level, latitude, longitude)**. Time values are monthly starting at 1987-6-1. There are 17 levels, the last level being 1000.0. The name of the vertical level axis is ‘level’. All the examples select the first two times and the last level. The last two examples remove the singleton level dimension from the result array.

In [ ]:
import cdms2
f = cdms2.open('ta_ncep_87-6-88-4.nc')
ta = f.variables['ta']

#### Keyword selection

In [ ]:
x = ta(time=('19988-1-1','1988-2-1'), level=1000.)
print(x.shape)

#### Interval indicator (see mapIntervalExt)

In [ ]:
x = ta(time=('1988-1-1','1988-3-1','co'), level=1000.)
print(x.shape)

##### Axis ID (plev) as a keyword

In [ ]:

x = ta(time=('1988-1-1','1988-2-1'), plev=1000.)
print(x.shape)

##### Positional

In [ ]:

x = ta(('1988-1-1','1988-2-1'),1000.0)
print(x.shape)

##### Predefined selectors

In [ ]:

from cdms2 import time, level
x = ta(time('1988-1-1','1988-2-1'), level(1000.))
print(x.shape)

from cdms2 import timeslice, levelslice
x = ta(timeslice(0,2), levelslice(16,17))
print(x.shape)

##### Call file as a function

In [ ]:

x = f('ta', time=('1988-1-1','1988-2-1'), level=1000.)
print(x.shape)

##### Python slices

In [ ]:
x = ta(time=slice(0,2), level=slice(16,17))
print(x.shape)

##### Selector objects

In [ ]:

from cdms2.selectors import Selector
sel = Selector(time=('1988-1-1','1988-2-1'), level=1000.)
x = ta(sel)
print(x.shape)
sel2 = Selector(time=('1988-1-1','1988-2-1'))
sel3 = sel2 & level(1000.0)
x = ta(sel3)
print(x.shape)
x = ta(sel2, level=1000.0)
print(x.shape)

##### Squeeze singleton dimension (level)

In [ ]:
x = ta[0:2,16]
print(x.shape)

x = ta(time=('1988-1-1','1988-2-1'), level=1000., squeeze=1)
print(x.shape)

f.close()

## Examples


In this example, two datasets are opened, containing surface air temperature ``(‘tas’)`` and upper-air temperature ``(‘ta’)`` respectively. Surface air temperature is a function of (time, latitude, longitude). Upper-air temperature is a function of (time, level, latitude, longitude). Time is assumed to have a relative representation in the datasets (e.g., with units **‘months since basetime’**).

Data is extracted from both datasets for January of the first input year through December of the second input year. For each time and level, three quantities are calculated: slope, variance, and correlation. The results are written to a netCDF file. For brevity, the functions corrCoefSlope and removeSeasonalCycle are omitted.

In [ ]:
import cdms2
import MV2

# Calculate variance, slope, and correlation of
# surface air temperature with upper air temperature
# by level, and save to a netCDF file. 'pathTa' is the location of
# the file containing 'ta', 'pathTas' is the file with contains 'tas'.
# Data is extracted from January of year1 through December of year2.
def ccSlopeVarianceBySeasonFiltNet(pathTa, pathTas, month1, month2):

       # Open the files for ta and tas
       fta = cdms2.open(pathTa)
       ftas = cdms2.open(pathTas)

       #Get upper air temperature
       taObj = fta['ta']
       levs = taObj.getLevel()

       #Get the surface temperature for the closed interval [time1,time2]
       tas = ftas('tas', time=(month1, month2, 'cc'))

       # Allocate result arrays
       newaxes = taObj.getAxisList(omit='time')
       newshape = tuple([len(a) for a in newaxes])
       cc = MV2.zeros(newshape, typecode=MV2.float, axes=newaxes, id='correlation')
       b = MV2.zeros(newshape, typecode=MV2.float, axes=newaxes, id='slope')
       v = MV2.zeros(newshape, typecode=MV2.float, axes=newaxes, id='variance')

       # Remove seasonal cycle from surface air temperature
       tas = removeSeasonalCycle(tas)

       # For each level of air temperature, remove seasonal cycle
       # from upper air temperature, and calculate statistics
       for ilev in range(len(levs)):
           ta = taObj(time=(month1,month2,'cc'), \
                      level=slice(ilev, ilev+1), squeeze=1)
           ta = removeSeasonalCycle(ta)
           cc[ilev], b[ilev] = corrCoefSlope(tas ,ta)
           v[ilev] = MV.sum( ta**2 )/(1.0*ta.shape[0])

       # Write slope, correlation, and variance variables
       f = cdms2.open('CC_B_V_ALL.nc','w')
       f.title = filtered
       f.write(b)
       f.write(cc)
       f.write(v)
       f.close()

if __name__=='__main__':
       pathTa = 'ta_ncep_87-6-88-4.nc'
       pathTas = 'tas_mo.nc'
       # Process Jan80 through Dec81
       ccSlopeVarianceBySeasonFiltNet(pathTa,pathTas,'1991-01','1991-12')

Notes:

1.   Two modules are imported, "cdms", and "MV". "MV" implements
     arithmetic functions.
15.  "taObj" is a file (persistent) variable. At this point, no data has
     actually been read. This happens when the file variable is sliced, or
     when the subRegion function is called. levs is an axis.
20.  Calling the file like a function reads data for the given variable
     and time range. Note that month1 and month2 are time strings.
25.  In contrast to "taObj", the variables "cc:" b" and "v" are
     transient variables, not associated with a file. The assigned names
     are used when the variables are written.
34.  Another way to read data is to call the variable as a function. The
     squeeze option removes singleton axes, in this case the level axis.
43.  Write the data. Axis information is written automatically.
50.  This is the main routine of the script. "pathTa" and "pathTas"
     pathnames. Data is processed from January 1980 through December 1981.


2.13.2. Example 2

In the next example, the pointwise variance of a variable over time is calculated, for all times in a dataset. The name of the dataset and variable are entered, then the variance is calculated and plotted via the vcs module.

In [ ]:
#!/usr/bin/env python
#
# Calculates gridpoint total variance
# from an array of interest
#

import cdms
from MV import *

# Wait for return in an interactive window

def pause():
    print Hit return to continue: ,
    line = sys.stdin.readline()

1.      # Calculate pointwise variance of variable over time
# Returns the variance and the number of points
# for which the data is defined, for each grid point
def calcVar(x):
    # Check that the first axis is a time axis
    firstaxis = x.getAxis(0)
    if not firstaxis.isTime():
        raise 'First axis is not time, variable:', x.id

    n = count(x,0)
    sumxx = sum(x*x)
    sumx = sum(x)
    variance = (n*sumxx -(sumx * sumx))/(n * (n-1.))

    return variance, n

  if __name__=='__main__':
    import vcs, sys

    print 'Enter dataset path [/pcmdi/cdms/obs/erbs_mo.xml]: ',
    path = string.strip(sys.stdin.readline())
    if path=='': path='/pcmdi/cdms/obs/erbs_mo.xml'

2.  # Open the dataset
    dataset = cdms.open(path)

    # Select a variable from the dataset
    print 'Variables in file:',path
    varnames = dataset.variables.keys()
    varnames.sort()
    for varname in varnames:

        var = dataset.variables[varname]
        if hasattr(var,'long_name'):
            long_name = var.long_name
        elif hasattr(var,'title'):
            long_name = var.title
        else:
            long_name = '?'

    print '%-10s: %s'%(varname,long_name)
    print 'Select a variable: ',
3.  varname = string.strip(sys.stdin.readline())
    var = dataset(varname)
    dataset.close()

    # Calculate variance, count, and set attributes
    variance,n = calcVar(var)
    variance.id = 'variance_%s'%var.id
    n.id = 'count_%s'%var.id
    if hasattr(var,'units'):
        variance.units = '(%s)^2'%var.units

    # Plot variance
    w=vcs.init()
4.  w.plot(variance)
    pause()
    w.clear()
    w.plot(n)
    pause()
    w.clear()

The result of running this script is as follows:

In [ ]:
% calcVar.py
Enter dataset path [/pcmdi/cdms/sample/obs/erbs_mo.xml]:

Variables in file: /pcmdi/cdms/sample/obs/erbs_mo.xml
albt    : Albedo TOA [%]
albtcs : Albedo TOA clear sky [%]
rlcrft  : LW Cloud Radiation Forcing TOA [W/m^2]
rlut    : LW radiation TOA (OLR) [W/m^2]
rlutcs : LW radiation upward TOA clear sky [W/m^2]
rscrft : SW Cloud Radiation Forcing TOA [W/m^2]
rsdt    : SW radiation downward TOA [W/m^2]
rsut    : SW radiation upward TOA [W/m^2]
rsutcs : SW radiation upward TOA clear sky [W/m^2]
Select a variable: albt

<The variance is plotted>

Hit return to continue:

<The number of points is plotted>

Notes:

    n = count(x, 0) returns the pointwise number of valid values, summing across axis 0, the first axis. count is an MV function.
    dataset is a Dataset or CdmsFile object, depending on whether a .xml or .nc pathname is entered. dataset.variables is a dictionary mapping variable name to file variable.
    var is a transient variable.
    Plot the variance and count variables. Spatial longitude and latitude information are carried with the computations, so the continents are plotted correctly.


In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()